In [ ]:
import torch
import numpy as np
import os
import sklearn.metrics

from model.unet_model import UNet
from visualisation import plot_precision_recall_f1, show_full_comparisonTestGenerator

from load_data import *
from post_processing import *

In [ ]:
# All the metrics available for testing.
METRICS = {
    "accuracy": sklearn.metrics.accuracy_score,
    "balanced_accuracy": sklearn.metrics.balanced_accuracy_score,
    "average_precision": sklearn.metrics.average_precision_score,
    "neg_brier_score": sklearn.metrics.brier_score_loss,
    "f1": sklearn.metrics.f1_score,
    "f1_weighted": lambda true, pred: sklearn.metrics.f1_score(
        true, pred, zero_division=1, average="weighted"
    ),
    "neg_log_loss": sklearn.metrics.log_loss,
    "precision": sklearn.metrics.precision_score,
    "precision_weighted": lambda true, pred: sklearn.metrics.precision_score(
        true, pred, zero_division=1, average="weighted"
    ),
    "recall": sklearn.metrics.recall_score,
    "recall_weighted": lambda true, pred: sklearn.metrics.recall_score(
        true, pred, zero_division=1, average="weighted"
    ),
    "jaccard": sklearn.metrics.jaccard_score,
    "jaccard_weighted": lambda true, pred: sklearn.metrics.jaccard_score(
        true, pred, average="weighted"
    ),
    "roc_auc": sklearn.metrics.roc_auc_score,
    "roc_auc_ovr" : sklearn.metrics.roc_auc_score,
    "roc_auc_ovo" : sklearn.metrics.roc_auc_score,
    "roc_auc_ovr_weighted" : sklearn.metrics.roc_auc_score,
    "roc_auc_ovo_weighted" : sklearn.metrics.roc_auc_score,
}

In [ ]:
def main(
    model_name,
    from_file=True,
    to_file=False,
    validation=True,
    test=["precision", "recall", "f1", "accuracy", "jaccard"],
    concat=False,
    plot=True,
):
    """
    Inputs:
    ========
    model_name : str
        Which model to do things with. This is assumed to be both the name of the directory in which parameters are stored, and the name of the parameters file.
    from_file : bool
        Whether data should be loaded from a file; otherwise it will be generated.
        The file should:
            - be in the same directory as the model parameters
            - be called "data.npz"
            - contain 4 arrays: "val_predictions", "val_labels", "test_predictions" and "test_labels".
        Labels are expected to be floats and will be thresholded.
        Predictions are expected to be raw (not probabilities).
    to_file : bool
        If data is generated (not loaded from a file), whether to save to a file in the model directory, according to the form described in from_file.
        Irrelevant when from_file is set to True.
    validation : bool
        Whether to go through validation steps (to find the best threshold).
    test : list of str
        Which metrics to use for testing (evaluate the model with a given threshold).
        The results are stored in a txt file called "test_results.txt" in the model directory.
        If empty then testing is skipped.
    concat : bool
        During testing, whether to compute each metric once, on the concatenation of the whole test set.
    plot : bool
        Whether to show plots during run.
    """
    model_dir = os.path.join(dir_models, model_name)
    params_file = os.path.join(model_dir, model_name)
    new_section = "=" * 50

    print("Importing model parameters from {}".format(params_file))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(n_channels=3, n_classes=1, bilinear=False)
    model = model.to(device)
    model.load_state_dict(torch.load(params_file, map_location=torch.device("cpu")))

    # File where data are stored, or will be if there aren't already
    data_file = os.path.join(model_dir, "data.npz")
    print(new_section)
    if from_file:
        print("Loading data")
        arrays = np.load(data_file)
        val_predictions, val_labels, test_predictions, test_labels = arrays.values()
    else:
        print("Generating data")
        dir_data_validation = os.path.join(dir_data, "validation")
        dir_data_test = os.path.join(dir_data, "test")

        _, validation_dl, test_dl = load_data(
            dir_data_validation=dir_data_validation,
            dir_data_test=dir_data_test,
            prop_noPV_training=0,  # Has no impact
            min_rescale_images=0,  # Has no impact
            batch_size=100,  # All of them
        )

        model.eval()
        with torch.no_grad():
            # Get images and labels from both DataLoaders
            val_images, val_labels = next(iter(validation_dl))
            test_images, test_labels = next(iter(test_dl))
            val_images = val_images.to(device, dtype=torch.float32)
            test_images = test_images.to(device, dtype=torch.float32)
            # Make predictions (predictions are not probabilities at this stage)
            print("Running model on data")
            val_predictions = model(val_images)
            test_predictions = model(test_images)
            # Convert to numpy arrays for computing
            val_predictions = np.squeeze(val_predictions.cpu().numpy())
            val_labels = np.squeeze(val_labels.cpu().numpy())
            test_predictions = np.squeeze(test_predictions.cpu().numpy())
            test_labels = np.squeeze(test_labels.cpu().numpy())
            # Save to file as numpy arrays
            if to_file:
                print("Saving results to file")
                np.savez_compressed(
                    data_file,
                    val_predictions=val_predictions,
                    val_labels=val_labels,
                    test_predictions=test_predictions,
                    test_labels=test_labels,
                )

    threshold_true = 0.5
    val_labels = np.where(val_labels > threshold_true, 1, 0)
    test_labels = np.where(test_labels > threshold_true, 1, 0)

    if validation:
        print(new_section)
        print("Validation starting")
        _, _, _, best_threshold = find_best_threshold(
            val_predictions, val_labels, n_thresholds=101, concat=concat, plot=plot
        )
        print(f"Found best threshold to be {best_threshold:.4f}")

    if test:
        print(new_section)
        print("Testing starting with metrics:")
        print(", ".join(test))
        results = test_model(
            test_predictions, test_labels, best_threshold, concat, *test
        )
        print(results)
        summary_type = "median"
        results_file = os.path.join(
            model_dir, "test_{}results.txt".format("concat_" if concat else "")
        )
        if concat:
            results_summary = np.transpose(results)
            print("Results:")
        else:
            results_summary = np.transpose(summary_stats(results, type=summary_type))
            print(f"Summary statistics are based on the {summary_type}")
            print("Results (lower, mid-point, upper):")
        print(f"\tBest threshold = {best_threshold:.4f}")
        for i, measure in enumerate(test):
            print("\t{}: {}".format(measure, results_summary[i, :]))
        print(new_section)
        print("Saving results to {}".format(results_file))
        np.savetxt(
            results_file,
            results_summary,
            fmt="%.4f",
            delimiter=" ",
            header=f"Threshold: {best_threshold:.3f}\n{'  '.join(test)}",
        )

    print("\n")
    
    
   

In [ ]:
 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(n_channels=3, n_classes=1, bilinear=False)
model = model.to(device)
model.load_state_dict(torch.load("../saved_models/6Adam_e_3_25noPV_BCEwithweights_epochs_80_schedulere_4_at50"))
# Visualise predictions of the model on individual images    
vizualization_generator = show_full_comparisonTestGenerator(model,threshold_prediction=0.79)
SHOW_N_IMAGES = 15
count = 0
for _ in vizualization_generator:
    count +=1
    if count >= SHOW_N_IMAGES:
        break